## Code for replicating gao et al research on VUA Sequence Model

In [2]:
# mount drive
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# add repo directory to path
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Repos/metaphor-detection']


In [4]:
# directories
# to download glove and elmo vectors see: notebooks/Download_large_data.ipynb
data_dir = repo_dir + '/resources/metaphor-in-context/data/'
glove_dir = repo_dir + '/resources/glove/'
elmo_dir = repo_dir + '/resources/elmo/'

In [9]:
# cd to working directory here if neccesary
%cd $repo_dir
%ls

/content/drive/MyDrive/Repos/metaphor-detection
core/  gao-g-requirements.txt  install.sh  notebooks/  README.md  resources/


In [10]:
# pip install requirements (takes a while)
!cd drive/MyDrive/Repos/metaphor-detection/; pip install -r gao-g-requirements.txt
!pip install --upgrade google-cloud-storage

/bin/bash: line 0: cd: drive/MyDrive/Repos/metaphor-detection/: No such file or directory
     |████████████████████████████████| 718 kB 12.8 MB/s 
     |████████████████████████████████| 46 kB 2.5 MB/s 
     |████████████████████████████████| 592 kB 60.0 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 240 kB 61.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 62.3 MB/s 
     |████████████████████████████████| 1.7 MB 60.2 MB/s 
     |████████████████████████████████| 312 kB 30.0 MB/s 
     |████████████████████████████████| 1.2 MB 68.5 MB/s 
     |████████████████████████████████| 3.5 MB 77.3 MB/s 
     |████████████████████████████████| 132 kB 64.3 MB/s 
     |████████████████████████████████| 8.6 MB 73.3 MB/s 
     |████████████████████████████████| 79 kB 6.1

In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
from core.gao_files.classification.util import get_num_lines, get_vocab, embed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix
from core.gao_files.classification.util import TextDatasetWithGloveElmoSuffix as TextDataset
from core.gao_files.classification.util import evaluate
from core.gao_files.classification.model import RNNSequenceClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader


import csv
import h5py
import random
import math
import numpy as np
import pandas as pd

In [13]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = True

PyTorch version:
1.10.0+cu111
GPU Detected:
False


In [14]:
!git statusgit add notebooks/

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   notebooks/sequence/replicate_trofi_seq_model_for_classification.ipynb
	modified:   notebooks/sequence/replicate_vua_seq_model_for_classification.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	notebooks/classification/replicate_trofi_cls_model_for_classification.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
